In [ ]:
# 한글 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os

# 구글 드라이브 연결
drive.mount('/content/drive')

In [ ]:
import os, glob, ast, json
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

TOKENIZED_DIR = "/content/drive/MyDrive/project/리뷰DB_tokenized"
OUT_DIR       = "/content/drive/MyDrive/project/리뷰DB_scored"
os.makedirs(OUT_DIR, exist_ok=True)

SENT_TOKEN_COL = "sentiment_tokens"
LEXICON_JSON_PATH = "/content/drive/MyDrive/project/SentiWord_info.json" # KNU

# 부정/강조
USE_RULES = True
NEGATORS = {"안", "못"}
DOWNERS  = {"별로"}
BOOSTERS = {"너무", "진짜", "완전"}

def to_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    if isinstance(x, str):
        try:
            v = ast.literal_eval(x)
            return v if isinstance(v, list) else []
        except:
            return [t.strip() for t in x.split(",") if t.strip()]
    return []

# 감성사전 로드: 단어(1그램) / 표현(2~N그램) 분리 (word_root 기준)
with open(LEXICON_JSON_PATH, "r", encoding="utf-8") as f:
    lex_data = json.load(f)

single_lex = {}
phrase_lex_by_len = defaultdict(dict)

for item in lex_data:
    root = str(item.get("word_root", "")).strip()
    if not root:
        continue

    pol = int(str(item.get("polarity", "0")).strip())
    toks = tuple(root.split())

    if len(toks) == 1:
        single_lex[toks[0]] = pol
    else:
        phrase_lex_by_len[len(toks)][toks] = pol

max_phrase_len = max(phrase_lex_by_len.keys(), default=1)

# ngram우선 매칭 + 부정/강조 룰
def find_phrase(tokens, start_idx):
    n = len(tokens)
    maxL = min(max_phrase_len, n - start_idx)
    for L in range(maxL, 1, -1):
        key = tuple(tokens[start_idx:start_idx+L])
        d = phrase_lex_by_len.get(L)
        if d and key in d:
            return L, d[key], " ".join(key)
    return 0, 0, ""

def score_tokens_ngram(tokens):
    """
    tokens: sentiment_tokens(form) 리스트
    return: total_score, matched_cnt, pos_cnt, neg_cnt, matched_terms
    """
    total = 0
    matched_cnt = 0
    pos_cnt = 0
    neg_cnt = 0
    matched_terms = []

    i = 0
    n = len(tokens)

    while i < n:
        w = tokens[i]

        # ---- (A) 부정/강조 단어가 있으면, 다음 구간을 '표현 우선'으로 찾아 적용 ----
        if USE_RULES and w in (NEGATORS | DOWNERS | BOOSTERS) and i + 1 < n:
            L, s, txt = find_phrase(tokens, i + 1)
            if L == 0:
                nxt = tokens[i + 1]
                s = single_lex.get(nxt, 0)
                txt = nxt
                L = 1

            if s != 0:
                if w in NEGATORS:
                    s2 = int(round(-0.8 * s))  # 반전+감쇠
                    total += s2
                    matched_cnt += 1
                    pos_cnt += 1 if s2 > 0 else 0
                    neg_cnt += 1 if s2 < 0 else 0
                    matched_terms.append((f"{w} {txt}", s2))
                    i += 1 + L
                    continue

                if w in DOWNERS:
                    s2 = -max(1, abs(s))       # '별로'는 부정 강화
                    total += s2
                    matched_cnt += 1
                    neg_cnt += 1
                    matched_terms.append((f"{w} {txt}", s2))
                    i += 1 + L
                    continue

                if w in BOOSTERS:
                    s2 = 2 if s > 0 else -2    # 강도 강화(단순 버전)
                    total += s2
                    matched_cnt += 1
                    pos_cnt += 1 if s2 > 0 else 0
                    neg_cnt += 1 if s2 < 0 else 0
                    matched_terms.append((f"{w} {txt}", s2))
                    i += 1 + L
                    continue

        # ---- (B) 일반: 표현(ngram) 우선 매칭 ----
        L, s, txt = find_phrase(tokens, i)
        if L > 0:
            total += s
            matched_cnt += 1
            pos_cnt += 1 if s > 0 else 0
            neg_cnt += 1 if s < 0 else 0
            matched_terms.append((txt, s))
            i += L
            continue

        # ---- (C) 단어(1그램) 매칭 ----
        s = single_lex.get(w, 0)
        if s != 0:
            total += s
            matched_cnt += 1
            pos_cnt += 1 if s > 0 else 0
            neg_cnt += 1 if s < 0 else 0
            matched_terms.append((w, s))

        i += 1

    return total, matched_cnt, pos_cnt, neg_cnt, matched_terms

files = sorted(glob.glob(os.path.join(TOKENIZED_DIR, "*.csv")))
print("📄 처리할 파일 수:", len(files))

summary_rows = []

for path in tqdm(files, desc="Scoring CSVs"):
    # 읽기
    try:
        df = pd.read_csv(path)
    except Exception:
        df = pd.read_csv(path, encoding="cp949")

    if SENT_TOKEN_COL not in df.columns:
        out_path = os.path.join(OUT_DIR, os.path.basename(path))
        df.to_csv(out_path, index=False, encoding="utf-8-sig")
        continue

    # 리스트 복원
    df[SENT_TOKEN_COL] = df[SENT_TOKEN_COL].apply(to_list)

    # 스코어링
    scored = df[SENT_TOKEN_COL].apply(score_tokens_ngram)
    df["sent_score"] = [x[0] for x in scored]
    df["sent_matched_cnt"] = [x[1] for x in scored]
    df["sent_pos_cnt"] = [x[2] for x in scored]
    df["sent_neg_cnt"] = [x[3] for x in scored]
    df["sent_matched_terms"] = [x[4] for x in scored]

    # 매칭률
    df["sent_len"] = df[SENT_TOKEN_COL].apply(len)
    df["sent_match_rate"] = df.apply(
        lambda r: (r["sent_matched_cnt"] / r["sent_len"]) if r["sent_len"] > 0 else 0,
        axis=1
    )

    # 2진 라벨: 긍=1, 부=0
    df["sent_binary"] = (df["sent_score"] > 0).astype(int)

    out_path = os.path.join(OUT_DIR, os.path.basename(path))
    df.to_csv(out_path, index=False, encoding="utf-8-sig")

    summary_rows.append({
        "file": os.path.basename(path),
        "n_rows": len(df),
        "avg_score": df["sent_score"].mean(),
        "pos_ratio(=1)": df["sent_binary"].mean(),         # 1의 비율
        "neg_ratio(=0)": 1 - df["sent_binary"].mean(),     # 0의 비율
        "avg_match_rate": df["sent_match_rate"].mean()
    })

summary = pd.DataFrame(summary_rows).sort_values("avg_score", ascending=False)
summary_path = os.path.join(OUT_DIR, "_summary_by_file.csv")
summary.to_csv(summary_path, index=False, encoding="utf-8-sig")

print("\n✅ 완료!")
print("📁 결과 폴더:", OUT_DIR)
print("📄 요약 파일:", summary_path)
display(summary.head(20))
